In [ ]:
import numpy as np
import os
from matplotlib import image
import matplotlib.pyplot as plt
import cv2 as cv

from terrain_nerf.autonav import arc

# autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
poses = np.load('../data/poses_20230625_1236.npz')['poses']

# Plot poses
plt.figure(figsize=(10, 10))
plt.plot(poses[:, 0], poses[:, 1], 'k.')
plt.axis('equal')
plt.show()

In [ ]:
from terrain_nerf.utils import trajectory_plot_trace
import plotly.graph_objects as go

transforms = np.load('transforms.npy')

In [ ]:
N = len(transforms)
Rs = np.zeros((3, 3, N))
ts = np.zeros((N, 3))
for i in range(N):
    Rs[:, :, i] = transforms[i][:3, :3]
    ts[i, :] = transforms[i][:3, 3]

# Plot trajectory
fig = go.Figure(data=trajectory_plot_trace(Rs, ts, scale=0.1))
# fig.add_trace(trajectory_plot_trace(Rs, ts, scale=1.0))
fig.update_layout(width=1500, height=900, scene_aspectmode='data')
fig.show()